In [ ]:
import geopandas as gpd



#Directory CHM france 2018, 2019, 2020
chm_2020 = '/Users/arthurcalvi/Data/Disturbances_maps/FORMS/2020_l93.tif'

#Directory folder with the tiles
tiles = '/Users/arthurcalvi/Data/species/validation/tiles'

# Load the data
result_gdf = gpd.read_parquet("/Users/arthurcalvi/Data/species/validation/val_train_tiles.parquet")
print(result_gdf.crs)



In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling, transform_geom
import os
import numpy as np
from shapely.geometry import box, mapping
from tqdm import tqdm

def crop_and_save_chm(chm_path: str, tiles_path: str, gdf: gpd.GeoDataFrame):
    """
    Crop the CHM for each tile using the geometry of the corresponding RGB file and save it in the corresponding folder.

    Args:
        chm_path (str): Path to the CHM .tif file.
        tiles_path (str): Path to the directory containing the tile folders.
        gdf (gpd.GeoDataFrame): GeoDataFrame containing the AOIs.
    """
    with rasterio.open(chm_path) as src:
        chm_crs = src.crs
        # Reproject gdf to match the CRS of the CHM
        gdf = gdf.to_crs(chm_crs)

        for idx, row in tqdm(gdf.iterrows()):
            # Match the correct folder
            tile_folders = sorted([f for f in os.listdir(tiles_path) if f.startswith(f'tile_') and f.split('_')[1].isdigit()], key=lambda x: int(x.split('_')[1]))
            matching_tile_folder = next((f for f in tile_folders if int(f.split('_')[1]) == idx), None)
            if matching_tile_folder:
                tile_folder = os.path.join(tiles_path, matching_tile_folder, 'tree_map')
                os.makedirs(tile_folder, exist_ok=True)
                output_path = os.path.join(tile_folder, 'CHM2020.tif')

                print(f"writing {output_path} for index {idx}")
                
                # Use the geometry of the RGB file for cropping
                rgb_folder = os.path.join(tiles_path, matching_tile_folder, 'rgb')
                for file in os.listdir(rgb_folder):
                    if file.endswith(".tif"):
                        with rasterio.open(os.path.join(rgb_folder, file)) as rgb_src:
                            target_crs = rgb_src.crs
                            target_bounds = rgb_src.bounds
                            target_geometry = box(target_bounds.left, target_bounds.bottom, target_bounds.right, target_bounds.top)
                            target_geometry = mapping(target_geometry)
                            target_transform = rgb_src.transform
                            target_width = rgb_src.width
                            target_height = rgb_src.height
                            # Transform the geometry to the CRS of the CHM
                            if target_crs != chm_crs:
                                target_geometry = transform_geom(target_crs, chm_crs, target_geometry)
                            break
                
                # Crop using the target geometry
                out_image, out_transform = mask(src, [target_geometry], crop=True)
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,
                    "crs": chm_crs,
                    "count": 1,
                    "nodata": src.nodata
                })
                
                if chm_crs != target_crs:

                    out_meta.update({
                        "crs": target_crs,
                        "transform": target_transform,
                        "width": target_width,
                        "height": target_height,
                        "nodata": src.nodata
                    })
                    with rasterio.open(output_path, "w", **out_meta) as dst:
                        reproject(
                            source=out_image,
                            destination=rasterio.band(dst, 1),
                            src_transform=out_transform,
                            src_crs=chm_crs,
                            dst_transform=target_transform,
                            dst_crs=target_crs,
                            resampling=Resampling.bilinear
                        )
                else:
                    with rasterio.open(output_path, "w", **out_meta) as dst:
                        dst.write(out_image[0], 1)


# Directory CHM France 2020
chm_2020 = '/Users/arthurcalvi/Data/Disturbances_maps/FORMS/2020_l93.tif'

# Directory folder with the tiles
tiles = '/Users/arthurcalvi/Data/species/validation/tiles'

# Load the data
result_gdf = gpd.read_parquet("/Users/arthurcalvi/Data/species/validation/val_train_tiles.parquet")
print(result_gdf.crs)

crop_and_save_chm(chm_2020, tiles, result_gdf)
